# Lets Begin

## Decrypt the current data

In [ ]:
from cryptography.fernet import Fernet
import os

folio_key = os.getenv("FOLIO_KEY")

f = Fernet(folio_key)

encrypt_dir = "current_data"
decrypt_dir = "current_data"

# Ensure the encrypt directory exists
os.makedirs(decrypt_dir, exist_ok=True)

# Encrypt each file in the current_data directory
for filename in os.listdir(encrypt_dir):
    file_path = os.path.join(encrypt_dir, filename)
    
    # Read the file data
    with open(file_path, "rb") as file:
        file_data = file.read()

    # Decrypt data
    encrypted_data = f.decrypt(file_data)

    # Write the encrypted file to the encrypt directory
    encrypted_file_path = os.path.join(decrypt_dir, filename)
    with open(encrypted_file_path, "wb") as file:
        file.write(encrypted_data)

print("Decryptiong complete")

Decryptiong complete


## Ingest new IKBR CSV report
Make sure to add the report into IB_export folder and name it with ib_DDMMYYYY.csv. Replace the value of `ib_extract_file_name` with the name of the file you just added. 

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime

%matplotlib inline
%config InlineBackend.figure_format ='retina'

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

ib_extract_file_name = "ib_19042025.csv"
source_folder = "IB_extract"
target_folder = "current_data"

"""
Function to Extract respective sections from IB extract csv file
"""
def extract(section_name, section_description):
    with open(f'{source_folder}/{ib_extract_file_name}', 'r') as source_file:
        lines = source_file.readlines()

    start_index = None
    end_index = None
    bos = f'"BOS","{section_name}","{section_description}"'
    eos = f'"EOS","{section_name}"'

    for i, line in enumerate(lines):
        if bos in line:
            start_index = i + 1
        if eos in line and start_index is not None:
            end_index = i - 1
            break

    if start_index is not None and end_index is not None:
        with open(f'{source_folder}/temp.csv', 'w') as positions_file:
            for line in lines[start_index:end_index + 1]:
                positions_file.write(line)

### Extract Cash Report (CRTT)

In [4]:
section_name = "CRTT"
extract(section_name, "Cash Report; trade date basis")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "LevelOfDetail",
        "FromDate", 
        "ToDate", 
        "Commissions",
        "Deposits",
        "Withdrawals",
        "BrokerInterest",
        "NetTradesSales",
        "NetTradesPurchases",
        "SalesTax",
        "EndingCash",
        "EndingSettledCash",
        "NetCashBalanceSLB",
        "NetSettledCashBalanceSLB",
    ]

data = pd.read_csv(f'{source_folder}/temp.csv')
cash_report = data[columns].copy()
cash_report.to_csv(f'{target_folder}/{section_name}.csv', index=False)
cash_report.head()

,ClientAccountID,CurrencyPrimary,LevelOfDetail,FromDate,ToDate,Commissions,Deposits,Withdrawals,BrokerInterest,NetTradesSales,NetTradesPurchases,SalesTax,EndingCash,EndingSettledCash,NetCashBalanceSLB,NetSettledCashBalanceSLB
0,U12283810,BASE_SUMMARY,BaseCurrency,20240418,20250417,-39.118092,82970.81,0,117.83,53887.525,-131227.28,-3.60614,5848.200346,5848.200346,5848.200346,5848.200346
1,U12283810,SGD,Currency,20240418,20250417,0.000000,0.00,0,0.00,0.000,0.00,0.00000,0.000059,0.000059,0.000059,0.000059
2,U12283810,USD,Currency,20240418,20250417,-39.118092,82970.81,0,117.83,53887.525,-131227.28,-3.60614,5848.200301,5848.200301,5848.200301,5848.200301


### Extract Cash Transactions (CTRN)

In [5]:
section_name = "CTRN"
extract(section_name, "Cash Transactions")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "FXRateToBase",
        "Date/Time", 
        "SettleDate", 
        "Amount",
        "Type",
        "TransactionID",
        "ReportDate",
    ]
data = pd.read_csv(f'{source_folder}/temp.csv')
source_ctrn = data[columns].copy()

data = pd.read_csv(f'{target_folder}/{section_name}.csv')
current_ctrn = data[columns].copy()

# Find TransactionIDs that are in source_ctrn but not in current_ctrn
new_trades = source_ctrn[~source_ctrn['TransactionID'].isin(current_ctrn['TransactionID'])]

# Append new transactions to current_ctrn
updated_trades = pd.concat([current_ctrn, new_trades])

# Write the updated dataframe to final.csv
updated_trades.to_csv(f'{target_folder}/{section_name}.csv', index=False)

### Positions (POST)

In [6]:
section_name = "POST"
extract(section_name, "Position; trade date basis")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "AssetClass",
        "Symbol", 
        "Description", 
        "Conid",
        "SecurityID",
        "ListingExchange",
        "IssuerCountryCode",
        "ReportDate",
        "Quantity",
        "CostBasisPrice",
        "CostBasisMoney",
        "PercentOfNAV",
    ]
data = pd.read_csv(f'{source_folder}/temp.csv')
positions = data[columns].copy()
positions.to_csv(f'{target_folder}/{section_name}.csv', index=False)
positions.head(20)

,ClientAccountID,CurrencyPrimary,AssetClass,Symbol,Description,Conid,SecurityID,ListingExchange,IssuerCountryCode,ReportDate,Quantity,CostBasisPrice,CostBasisMoney,PercentOfNAV
0,U12283810,USD,STK,ALAB,ASTERA LABS INC,692196414,US04626A1034,NASDAQ,US,20250417,40,66.413675,2656.547008,2.14
1,U12283810,USD,STK,AMD,ADVANCED MICRO DEVICES,4391,US0079031078,NASDAQ,US,20250417,176,124.038660,21830.804222,13.94
2,U12283810,USD,STK,BITB,BITWISE BITCOIN ETF,677037658,US09174C1045,ARCA,US,20250417,200,45.823284,9164.656881,8.37
3,U12283810,USD,STK,BTDR,BITDEER TECHNOLOGIES GROUP-A,625545608,KYG114481008,NASDAQ,SG,20250417,1495,8.191621,12246.473390,10.31
4,U12283810,USD,STK,CLSK,CLEANSPARK INC,395179962,US18452B2097,NASDAQ,US,20250417,200,11.697072,2339.414324,1.36
5,U12283810,USD,STK,IRD,OPUS GENETICS INC,453924737,US67577R1023,NASDAQ,US,20250417,5500,1.378996,7584.477624,3.78
6,U12283810,USD,STK,IREN,IREN LTD,526906130,AU0000185993,NASDAQ,AU,20250417,750,4.396856,3297.642031,3.80
7,U12283810,USD,STK,LFMD,LIFEMD INC,472693173,US53216B1044,NASDAQ,US,20250417,330,5.142054,1696.877885,1.62
8,U12283810,USD,STK,MRK,MERCK & CO INC,70101545,US58933Y1055,NYSE,US,20250417,50,99.855700,4992.785013,3.53
9,U12283810,USD,STK,MU,MICRON TECHNOLOGY INC,9939,US5951121038,NASDAQ,US,20250417,90,94.381152,8494.303709,5.61


### Trades (TRNT)

In [7]:
section_name = "TRNT"
extract(section_name, "Trades; trade date basis")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "AssetClass",
        "Symbol", 
        "Description", 
        "Conid",
        "SecurityID",
        "ListingExchange",
        "IssuerCountryCode",
        "TradeID",
        "TradeDate",
        "SettleDateTarget",
        "TransactionType",
        "Exchange",
        "Quantity",
        "TradePrice",
        "TradeMoney",
        "Taxes",
        "IBCommission",
        "NetCash",
        "CostBasis",
        "FifoPnlRealized",
        "Buy/Sell",
        "LevelOfDetail",
    ]
data = pd.read_csv(f'{source_folder}/temp.csv')
trades = data[columns].copy()

# Fetch current trades snapshot data
data = pd.read_csv(f'{target_folder}/{section_name}.csv')
current_trades = data[columns].copy()

# Find records that are in source trades but not in current_trades snapshot
new_trades = trades[~trades.apply(tuple,1).isin(current_trades.apply(tuple,1))]

# Append new transactions to current_trades
updated_trades = pd.concat([current_trades, new_trades])

# # Write the updated dataframe to final.csv
updated_trades.to_csv(f'{target_folder}/{section_name}.csv', index=False)
updated_trades.head(10)



,ClientAccountID,CurrencyPrimary,AssetClass,Symbol,Description,Conid,SecurityID,ListingExchange,IssuerCountryCode,TradeID,...,Quantity,TradePrice,TradeMoney,Taxes,IBCommission,NetCash,CostBasis,FifoPnlRealized,Buy/Sell,LevelOfDetail
0,U12283810,USD,STK,ADBE,ADOBE INC,265768,US00724F1012,NASDAQ,US,NaN,...,16.0,510.000000,8160.0,-0.028931,-0.321457,-8160.350388,8160.350388,0.000000,BUY,SYMBOL_SUMMARY
1,U12283810,USD,STK,ADBE,ADOBE INC,265768,US00724F1012,NASDAQ,US,NaN,...,-16.0,506.500000,-8104.0,-0.034861,-0.387345,8103.577794,-8160.350388,-56.772594,SELL,SYMBOL_SUMMARY
2,U12283810,USD,STK,ADBE,ADOBE INC,265768,US00724F1012,NASDAQ,US,NaN,...,16.0,510.000000,8160.0,-0.028931,-0.321457,-8160.350388,8160.350388,0.000000,BUY,ORDER
3,U12283810,USD,STK,ADBE,ADOBE INC,265768,US00724F1012,NASDAQ,US,31291086.0,...,16.0,510.000000,8160.0,-0.028931,-0.321457,-8160.350388,8160.350388,0.000000,BUY,EXECUTION
4,U12283810,USD,STK,ADBE,ADOBE INC,265768,US00724F1012,NASDAQ,US,NaN,...,-16.0,506.500000,-8104.0,-0.034861,-0.387345,8103.577794,-8160.350388,-56.772594,SELL,ORDER
5,U12283810,USD,STK,ADBE,ADOBE INC,265768,US00724F1012,NASDAQ,US,31630343.0,...,-16.0,506.500000,-8104.0,-0.034861,-0.387345,8103.577794,-8160.350388,-56.772594,SELL,EXECUTION
6,U12283810,USD,STK,ADBE,ADOBE INC,265768,US00724F1012,NASDAQ,US,NaN,...,16.0,510.021899,NaN,NaN,NaN,NaN,8160.350388,-56.772594,SELL,CLOSED_LOT
7,U12283810,USD,STK,AMD,ADVANCED MICRO DEVICES,4391,US0079031078,NASDAQ,US,NaN,...,176.0,124.034091,21830.0,-0.079402,-0.724820,-21830.804221,21830.804221,0.000000,BUY,SYMBOL_SUMMARY
8,U12283810,USD,STK,AMD,ADVANCED MICRO DEVICES,4391,US0079031078,NASDAQ,US,NaN,...,100.0,125.300000,12530.0,-0.029023,-0.165057,-12530.194080,12530.194080,0.000000,BUY,ORDER
9,U12283810,USD,STK,AMD,ADVANCED MICRO DEVICES,4391,US0079031078,NASDAQ,US,47967000.0,...,15.0,125.300000,1879.5,-0.029023,-0.322477,-1879.851500,1879.851500,0.000000,BUY,EXECUTION


## Lock Files

In [ ]:
from cryptography.fernet import Fernet

# Create new key - YOU SHOULD NOT NEED THIS ANYMORE
# def write_key():
#     key = Fernet.generate_key()
#     with open("folio.key", "wb") as key_file:
#         key_file.write(key)

# write_key()

#### Encrypt data before code commit

In [8]:
from cryptography.fernet import Fernet
import os

folio_key = os.getenv("FOLIO_KEY")

f = Fernet(folio_key)

# Define the directories
current_data = "current_data"
IB_extract = "IB_extract"

def encrypt_file(directory, file_name):
    file_path = os.path.join(directory, file_name)
    with open(file_path, "rb") as file:
        file_data = file.read()

    encrypted_data = f.encrypt(file_data)

    with open(file_path, "wb") as file:
        file.write(encrypted_data)

# Encrypt each file in the current_data directory
for filename in os.listdir(current_data):
    encrypt_file(current_data, filename)

# Encrypt IB_extract report
encrypt_file(IB_extract, ib_extract_file_name)

# Encrypt temp file
encrypt_file(IB_extract, "temp.csv")

print("Encryption completed")



Encryption completed


#### Decrypt

In [ ]:
folio_key = os.getenv("FOLIO_KEY")

f = Fernet(folio_key)

encrypt_dir = "current_data"
decrypt_dir = "current_data"

# Ensure the encrypt directory exists
os.makedirs(decrypt_dir, exist_ok=True)

# Encrypt each file in the current_data directory
for filename in os.listdir(encrypt_dir):
    file_path = os.path.join(encrypt_dir, filename)
    
    # Read the file data
    with open(file_path, "rb") as file:
        file_data = file.read()

    # Decrypt data
    encrypted_data = f.decrypt(file_data)

    # Write the encrypted file to the encrypt directory
    encrypted_file_path = os.path.join(decrypt_dir, filename)
    with open(encrypted_file_path, "wb") as file:
        file.write(encrypted_data)

print("Decryptiong complete")